In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

In [3]:
df = pd.read_csv('/content/german_credit_augmented.csv',
                 parse_dates = ['contract_dt'])
con = sqlite3.connect('db')
df.to_sql('table1', con = con, index = False, if_exists = 'replace')

def select(sql):
    return pd.read_sql(sql, con)

In [4]:
display(df.head())
df.info()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   age               1000 non-null   int64         
 1   sex               1000 non-null   object        
 2   job               1000 non-null   int64         
 3   housing           1000 non-null   object        
 4   saving_accounts   817 non-null    object        
 5   checking_account  606 non-null    object        
 6   credit_amount     1000 non-null   int64         
 7   duration          1000 non-null   int64         
 8   purpose           1000 non-null   object        
 9   default           1000 non-null   int64         
 10  contract_dt       1000 non-null   datetime64[ns]
 11  client_id         1000 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(5)
memory usage: 93.9+ KB


## Первый запрос

Выведем количество кредитов, сгруппируем по целям. Нас интересуют только кредиты, взятые женщинами (sex = 'female'), которые владеют домом (housing = 'own'). Выведем только те категории, для которых количество кредитов больше, скажем, 12. Отсортируем по убыванию кол-ва кредитов.

In [5]:
sql = '''
SELECT
    purpose,
    COUNT(*) AS cnt
FROM
    table1
WHERE
    sex = 'female'
    AND housing = 'own'
GROUP BY
    purpose
HAVING
    cnt > 12
ORDER BY
    cnt DESC
'''

select(sql)

,purpose,cnt
0,radio/TV,68
1,car,53
2,furniture/equipment,38
3,education,15


Найдём средние значения взятых кредитов для каждого пола.

In [ ]:
sql = '''
SELECT
    sex,
    AVG(credit_amount) as avg_credit
FROM
    table1
GROUP BY
    sex
'''

select(sql)

,sex,avg_credit
0,female,2877.774194
1,male,3448.040580


Найдём среднее значение кредита для людей с разным количеством работ.

In [ ]:
sql = '''
SELECT
    job,
    AVG(credit_amount) as avg_credit
FROM
    table1
GROUP BY
    job
ORDER BY
    avg_credit DESC
'''

select(sql)

,job,avg_credit
0,3,5435.493243
1,2,3070.965079
2,0,2745.136364
3,1,2358.520000


Создадим сводную таблицу, в которой покажем, как связан тип собственности недвижимости с количеством клиентов с разбивкой по полу.

In [8]:
sql = '''
SELECT
    t.housing,
    COUNT(CASE WHEN t.sex = 'female' THEN 1 ELSE NULL END) AS female_cnt,
    COUNT(CASE WHEN t.sex = 'male' THEN 1 ELSE NULL END) AS male_cnt,
    COUNT(*) AS cnt
FROM
    table1 AS t
GROUP BY
    t.housing
'''

select(sql)

,housing,female_cnt,male_cnt,cnt
0,free,19,89,108
1,own,196,517,713
2,rent,95,84,179
